In [1]:
#General imports needed for path
import os 
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

#import for plots
import matplotlib.pyplot as plt

In [2]:
import route_dynamics.route_elevation.base_df as re_base
from route_dynamics.route_energy import knn
from route_dynamics.route_energy import constant_a as ca
from route_dynamics.route_riders import route_riders as ride 

import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping

In [3]:
#User defines what routes they want to evaluate
rt_list = [22]

#Import data file
shapefile_name = '../data/rt' + str(rt_list[0]) + '_pts.shp'

#Initialize route dataframe
rt_df = re_base.wrapper(shapefile_name)

In [4]:
rt_df.head()

,Z,geometry,length,grade
0,333.922952,POINT (-122.38813 47.56295),0,0.000000
1,333.782353,POINT (-122.38810 47.56295),6,-0.023433
2,333.642295,POINT (-122.38808 47.56295),12,-0.023343
3,333.502775,POINT (-122.38806 47.56295),18,-0.023253
4,333.363793,POINT (-122.38803 47.56295),24,-0.023164


In [18]:
# df, riders, mass = ride.route_ridership('AM', rt_list[0]) 

# stop_coord, rider_coord = ride.stop_coord(rt_list[0], riders)
# coords = rider_coord['coordinates'].values
# mass2 = rider_coord['Mean'].values

# stop_coords = stop_coord.coordinates.values

period = 'AM'
route = 22

In [19]:
trip183 = pd.read_csv("../data/Trip183.csv", usecols = ['SignRt', 'InOut', 'KeyTrip', 'BusType', 'Seats', 
                     'Period', 'AnnRides']) # KCM Data
trip183unsum = pd.read_csv("../data/Zon183Unsum.csv", usecols = ['Route', 'Dir', 'Trip_ID', 'InOut', 'STOP_SEQ', 'STOP_ID',
                     'Period', 'AveOn', 'AveOff', 'AveLd', 'Obs']) # KCM Data

In [20]:
bus_mass = {
    11: 11000,
    26: 19051,
    32: 11793,
    36: 11793,
    37: 12247,
    43: 14913,
    45: 19051,
    46: 18835,
    60: 19051,
    62: 19051,
    68: 19051,
    70: 12927,
    72: 12927,
    73: 12927,
    80: 19051,
    81: 19051,
    82: 19051,
    90: 12927,
    91: 12927,
    92: 12927,
    95: 19051,
    96: 19051
        }

trip183 = trip183.replace({'BusType': bus_mass})
trip_mass = trip183[['BusType', 'KeyTrip']]
trip_dict = dict(zip(trip_mass.KeyTrip, trip_mass.BusType))

In [21]:
df = trip183unsum
df = df.drop(df[(df.Period != period)].index)
df = df.drop(df[(df.Route != route)].index)

In [23]:
final_df = df.sort_values(by=['Trip_ID', 'STOP_SEQ', 'STOP_ID'])
seq_id = final_df[['STOP_SEQ', 'STOP_ID']]
seq_id2 = seq_id.drop_duplicates(subset=['STOP_SEQ'], keep='first')
seq_id3 = seq_id2.sort_values(by='STOP_SEQ')
stopid_dic = dict(zip(seq_id3.STOP_SEQ, seq_id3.STOP_ID))
riders = final_df.pivot(index='STOP_SEQ', columns='Trip_ID', values='AveLd')

keyfind = list(riders.columns)
mass_bus = [trip_dict[x] for x in keyfind]
mode = mode_mass = max(set(mass_bus), key=mass_bus.count)
rider_columns = list(riders.columns)
kept_columns = []

for i in range(0, len(mass_bus)):
        if mass_bus[i] == mode:
            kept_columns.append(rider_columns[i])

riders_interm = riders[kept_columns]
riders_kept = pd.DataFrame((riders_interm.mean(axis=1)), columns=['Mean'])
riders_kept.Mean*=80
riders_kept.reset_index(inplace=True)
riders_kept.replace({"STOP_SEQ": stopid_dic}, inplace=True)
riders_kept.columns = ['STOP_ID', 'Mean']

In [24]:
stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'

In [25]:
stops = gpd.read_file(stops_shp)
stops['ROUTE_LIST'].fillna(value=str(0), inplace=True)

In [26]:
stops_list = pd.DataFrame()
for i in range(0, len(stops)):

    if str(rt_list[0]) in (stops['ROUTE_LIST'][i]):
        for x in stops['ROUTE_LIST'][i].split(' '):
            if str(rt_list[0]) == x:
                stops_list = stops_list.append(stops.iloc[i])
            else:
                pass
    else:
        pass

stop_ids = stops_list['STOP_ID'].values

geometry = stops_list.geometry.values

xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy

xy_df['STOP_ID'] = stops_list['STOP_ID'].values

df = riders_kept
df_ind = df.reset_index()
df_ind.columns = ['STOP_SEQ', 'STOP_ID', 'Mean']

df_comb = xy_df.merge(df_ind, how='right', on="STOP_ID")
df_comb = df_comb.drop_duplicates(subset=['STOP_ID'], keep='first')
df_comb = df_comb[['STOP_SEQ', 'STOP_ID', 'coordinates', 'Mean']]
df_combine = df_comb.sort_values(by='STOP_SEQ')

df_combine.head()

,STOP_SEQ,STOP_ID,coordinates,Mean
112,0,22264.0,"(-122.37689915659708, 47.516854308421095)",91.2
22,1,22280.0,"(-122.37693810386416, 47.51379557835211)",91.2
24,2,22290.0,"(-122.3792737907224, 47.5138076827721)",115.2
26,3,22300.0,"(-122.38213977047363, 47.51382265450292)",217.6
28,4,22320.0,"(-122.38475476992342, 47.513836046877465)",235.2


In [29]:
stop_coords = df_combine.coordinates.values

In [30]:
# add stops to bus dataframe

stop_nn_indicies, stop_coord_nn = knn.find_knn(
                1,
                rt_df.geometry.values,
                stop_coords
                )

In [31]:
rt_df = rt_df.assign(
                is_bus_stop = ([False] * len(rt_df.index))
                )

for i in stop_nn_indicies.ravel():
    rt_df.at[i, 'is_bus_stop'] = True

In [32]:
true_count = rt_df.is_bus_stop.sum()
true_count

57

In [34]:
rt_df.head()

,Z,geometry,length,grade,is_bus_stop
0,333.922952,POINT (-122.388128652 47.56294666500003),0,0.000000,False
1,333.782353,POINT (-122.3881042931426 47.56294667086699),6,-0.023433,False
2,333.642295,POINT (-122.3880799342853 47.56294667673397),12,-0.023343,False
3,333.502775,POINT (-122.3880555754279 47.56294668260093),18,-0.023253,False
4,333.363793,POINT (-122.3880312165706 47.5629466884679),24,-0.023164,False


In [39]:
mass = mode
mass2 = df_combine.Mean.values
mass_array=mass2 + mass

unloaded_bus_mass = mass

In [40]:
# Initialize array of Nan's for mass column of rdf
full_mass_column = np.zeros(len(rt_df.index))
full_mass_column[:] = np.nan

# Iterate through the length of the given mass_array
# (already determined equal length to 'stop_coords').
for i in range(len(mass_array)):
    # Set values of mass at bus_stops
    full_mass_column[
        stop_nn_indicies[i]
        ] = mass_array[i]

# Set initial and value to unloaded bus mass.
full_mass_column[0] = unloaded_bus_mass
full_mass_column[-1] = unloaded_bus_mass

# Iterate through the half constructed rdf mass column
# ('full_mass_column') and fill in sapce between stops with previous value
for i in range(len(full_mass_column)-1):
    j = 1
    try:
        while np.isnan(full_mass_column[i+j]):
            full_mass_column[i+j] = full_mass_column[i]
            # print(full_mass_column[i+j] )
            j+=1
    except: IndexError

In [42]:
rt_df = rt_df.assign(
            mass = full_mass_column
            )

In [43]:
rt_df.head()

,Z,geometry,length,grade,is_bus_stop,mass
0,333.922952,POINT (-122.388128652 47.56294666500003),0,0.000000,False,12927.0
1,333.782353,POINT (-122.3881042931426 47.56294667086699),6,-0.023433,False,12927.0
2,333.642295,POINT (-122.3880799342853 47.56294667673397),12,-0.023343,False,12927.0
3,333.502775,POINT (-122.3880555754279 47.56294668260093),18,-0.023253,False,12927.0
4,333.363793,POINT (-122.3880312165706 47.5629466884679),24,-0.023164,False,12927.0
